In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark import SparkFiles


path = "../data/conditions_truncated.csv"

conf = SparkConf()
conf.getAll()

sc = SparkContext(appName="test")
    
spark = SparkSession(sc)
sc.setLogLevel("ERROR")

textfile = sc.textFile(path)

sc.addFile(path)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/09 11:52:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
SUPPORT = 1000
K = 2

def build_pairs(basket, filtered_diseases):
    """
    Returns a list of bigrams that only contain diseases from the filtered list.
    
    Parameters:
    - basket: a tuple representing a basket of items
    - filtered_diseases: a list of diseases above the support treshold to include in the bigrams
    
    Returns:
    - a list of bigrams, where each bigram is a tuple of two diseases
    """
    basket_diseases = basket[1] # list of diseases in the current basket
    bigrams = []

    for i, disease_1 in enumerate(basket_diseases):
        # if the current disease is not in the list of filtered diseases, skip
        if disease_1 not in filtered_diseases:
            continue

        for disease_2 in basket_diseases[i+1:]:
        # if the current disease is not in the list of filtered diseases, skip
            if disease_2 not in filtered_diseases:
                continue

            # create a bigram from the two diseases and append it to the list of bigrams
            bigram = f"{disease_1},{disease_2}"
            bigrams.append((bigram, 1))
    
    return bigrams



before_baskets =  textfile.map(lambda line: line.split(",")) \
                        .map(lambda pair: (pair[2],[pair[4]])) \
                            .reduceByKey(lambda a,b: a + b)

# TODO: remover repetidos: fazer set de doenças e ordernar
# TODO: checkar se isto está bem feito
baskets = before_baskets.mapValues(lambda x: sorted(set(x)))

printable = baskets.collect()

print(sc.parallelize(printable).count())

first_support = baskets.flatMap(lambda basket: [(disease, 1) for disease in basket[1]]) \
                            .reduceByKey(lambda a,b: a + b) \
                                .filter(lambda line: line[1] > SUPPORT)

printable2 = first_support.collect()

print(sc.parallelize(printable2).count())

filtered_diseases = first_support.map(lambda line: line[0]).collect()

bigrams_support = baskets.flatMap(lambda basket: build_pairs(basket, filtered_diseases)) \
                                .reduceByKey(lambda a,b: a + b) \
                                    .filter(lambda line: line[1] > SUPPORT)

printable3 = bigrams_support.sortBy(lambda line: line[1], False).collect()

for row in printable3:
    print(row)

print(len(printable3))
    

21574
32
('195662009,444814009', 6453)
('10509002,444814009', 5576)
('15777000,271737000', 5301)
('162864005,444814009', 4537)
('15777000,444814009', 4427)
('271737000,444814009', 4353)
('10509002,195662009', 3937)
('444814009,59621000', 3735)
('162864005,195662009', 3178)
('15777000,195662009', 3062)
('40055000,444814009', 3045)
('195662009,271737000', 3024)
('10509002,162864005', 2632)
('444814009,72892002', 2597)
('195662009,59621000', 2596)
('10509002,271737000', 2564)
('10509002,15777000', 2546)
('19169002,444814009', 2525)
('271737000,59621000', 2331)
('10509002,59621000', 2207)
('15777000,59621000', 2200)
('195662009,40055000', 2147)
('162864005,59621000', 2047)
('162864005,40055000', 1954)
('15777000,162864005', 1916)
('162864005,271737000', 1876)
('43878008,444814009', 1867)
('10509002,40055000', 1859)
('15777000,40055000', 1829)
('195662009,72892002', 1822)
('271737000,40055000', 1808)
('19169002,195662009', 1742)
('162864005,19169002', 1736)
('15777000,19169002', 1650)
('444

In [8]:
def build_trios(basket, filtered_diseases, filtered_diseases2):
    """
    Returns a list of trigrams that contain diseases from the filtered list.
    
    Parameters:
    - basket: a tuple representing a basket of items
    - filtered_diseases: a list of diseases above the support treshold to include in the trigrams
    - filtered_diseases2: a list of bigrams to include in the trigrams
    
    Returns:
    - a list of trigrams, where each trigram is a tuple of three diseases
    """
    diseases = basket[1] # list of diseases in the current basket
    trigrams = []
    
    for i, desease1 in enumerate(diseases):
        if desease1 not in filtered_diseases:
            continue
        for j, desease2 in enumerate(diseases[i+1:], i+1):
            # if the current disease is not in the list of filtered diseases, skip
            if desease2 not in filtered_diseases:
                continue

            # create a bigram from the two diseases and check if it is in the 
            # list of filtered bigrams
            if f"{desease1},{desease2}" not in filtered_diseases2:
                continue

            for desease3 in diseases[j+1:]:
                # if the current disease is not in the list of filtered diseases, 
                # skip to the next iteration
                if desease3 not in filtered_diseases:
                    continue
                
                # check if the two bigrams that can be created from the trigram 
                # are in the list of filtered bigrams
                if f"{desease1},{desease3}" not in filtered_diseases2 \
                    or f"{desease2},{desease3}" not in filtered_diseases2:
                    continue

                # create a trigram from the three diseases and append it to the 
                # list of trigrams
                trigrams.append((f"{desease1},{desease2},{desease3}", 1))
    return trigrams
    

filtered_diseases2 = bigrams_support.map(lambda line: line[0]).collect()

trigrams_support = baskets.flatMap(lambda basket: 
                                   build_trios(
                                        basket, 
                                        filtered_diseases, 
                                        filtered_diseases2)
                                        ).reduceByKey(lambda a,b: a + b) \
                                            .filter(lambda line: line[1] > SUPPORT)

printable4 = trigrams_support.sortBy(lambda line: line[1], False).collect()

for row in printable4:
    print(row)

print(len(printable4))

('72892002,72892002,72892002', 14568)
('444814009,444814009,444814009', 10451)
('72892002,72892002,444814009', 5195)
('15777000,271737000,444814009', 4905)
('72892002,444814009,72892002', 4712)
('162864005,444814009,444814009', 3898)
('15777000,444814009,444814009', 3690)
('59621000,72892002,72892002', 3626)
('444814009,72892002,72892002', 3548)
('444814009,444814009,195662009', 3525)
('271737000,444814009,444814009', 3348)
('59621000,444814009,444814009', 3156)
('444814009,195662009,444814009', 3073)
('72892002,72892002,195662009', 3026)
('72892002,195662009,72892002', 2776)
('15777000,271737000,195662009', 2733)
('444814009,444814009,10509002', 2657)
('195662009,444814009,444814009', 2526)
('72892002,72892002,10509002', 2504)
('40055000,444814009,444814009', 2443)
('444814009,10509002,444814009', 2318)
('19169002,444814009,444814009', 2241)
('72892002,10509002,72892002', 2228)
('19169002,72892002,72892002', 2186)
('15777000,271737000,10509002', 2124)
('195662009,72892002,72892002', 2

In [ ]:
sc.stop()